# Classifying 90+ Anime Characters with Transfer Learning


## Dataset Processing

First, install the prerequisite libraries:

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import os
import tensorflow as tf
import zipfile

Extract the dataset:

In [ ]:
base_dir = '/content/anime_dataset'

with zipfile.ZipFile("/content/drive/My Drive/dataset_cropped.zip") as datazip:
  datazip.extractall(base_dir)

train_dir = os.path.join(base_dir, 'cropped_images', 'train')
train_dir = os.path.join(base_dir, 'cropped_images', 'validation')

In [5]:
base_dir = '/content/anime_dataset'
train_dir = os.path.join(base_dir, 'cropped_images', 'train')
validation_dir = os.path.join(base_dir, 'cropped_images', 'validation')

Setup data augmentation/batch sizes for input:





In [6]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input, # Preprocess images for Inception v3
    rotation_range=30, # Randomly rotate the image up to 30 degrees
    horizontal_flip=True # Randomly flip the image horizontally
)

# No need for data augmentation in validation
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical', # Since we're classifying many different classes 
    batch_size=32, # Flow in batches of 32 images
    target_size=(299,299) # Reshape images to max image dimensions for Inceptionv3
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    class_mode='categorical',
    batch_size=32,
    target_size=(299,299)
)

Found 6341 images belonging to 94 classes.
Found 1063 images belonging to 94 classes.


Import and configure the base model (Inception V3):

In [7]:
base_model = tf.keras.applications.InceptionV3(
    include_top=False, # Don't include last output layer for transfer learning
    input_shape=(299,299,3),
    weights='imagenet' # Import the weights for imagenet
)

base_model.trainable = False # Freeze the layers so we're not retraining

87916544/87910968 [==============================] - 4s 0us/step


## Model Building

Create our own layers:


In [8]:
# Pool the 3D output into a 2D feature vector to connect conv layers to our fully connected layers
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D() 
fc_layer = tf.keras.layers.Dense(1024, activation='relu')
output_layer = tf.keras.layers.Dense(94, activation='softmax') # Since we have 94 classes

Build our model:

In [11]:
inputs = tf.keras.Input(shape=(299,299,3))
x = base_model(inputs, training=False) # We need training=False since we have batch normalization layers in Inception v3
x = global_avg_layer(x)
x = tf.keras.layers.Dropout(0.2)(x) # For regularization
x = fc_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = output_layer(x)

model = tf.keras.Model(inputs, outputs)

Compile our model:

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 94)               

## Training

Train the model:


In [14]:
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

In [ ]:
history = model.fit(
    x=train_generator,
    validation_data=validation_generator,
    epochs=100,
    steps_per_epoch=198 # Since we have 6341 training images and batch sizes of 32, after 198 batches we'll have gone through a little less than the whole training set
)

Epoch 1/100
198/198 [==============================] - 201s 1s/step - loss: 4.1059 - accuracy: 0.0910 - val_loss: 3.8914 - val_accuracy: 0.1656
Epoch 2/100
198/198 [==============================] - 199s 1s/step - loss: 3.7660 - accuracy: 0.1506 - val_loss: 3.6341 - val_accuracy: 0.1957
Epoch 3/100
198/198 [==============================] - 201s 1s/step - loss: 3.4918 - accuracy: 0.1973 - val_loss: 3.4116 - val_accuracy: 0.2342
Epoch 4/100
198/198 [==============================] - 201s 1s/step - loss: 3.2931 - accuracy: 0.2284 - val_loss: 3.2450 - val_accuracy: 0.2756
Epoch 5/100
198/198 [==============================] - 202s 1s/step - loss: 3.1048 - accuracy: 0.2634 - val_loss: 3.0919 - val_accuracy: 0.2926
Epoch 6/100
198/198 [==============================] - 202s 1s/step - loss: 2.9300 - accuracy: 0.2994 - val_loss: 2.9872 - val_accuracy: 0.3039
Epoch 7/100
198/198 [==============================] - 199s 1s/step - loss: 2.8119 - accuracy: 0.3143 - val_loss: 2.8762 - val_accuracy: